In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
import joblib


In [2]:
def load_data(filepath):
    data = pd.read_csv(filepath)
    return data

In [3]:
df = load_data('data/processed-data.csv')
print(df.shape)

(245955, 34)


In [4]:
df

,year,quarter,season,citymarketid_1,city1,airportid_1,airport_iata_1,airport_name_1,airport_name_concat_1,state_1,...,carrier_lg,carrier_lg_name,carrier_lg_name_concat,large_ms,fare_lg,carrier_low,carrier_low_name,carrier_low_name_concat,lf_ms,fare_low
0,2021,3,Summer,30135,"Allentown/Bethlehem/Easton, PA",10135,ABE,Lehigh Valley International Airport,ABE - Lehigh Valley International Airport,Pennsylvania,...,G4,Allegiant Air,G4 - Allegiant Air,1.0000,81.43,G4,Allegiant Air,G4 - Allegiant Air,1.0000,81.43
1,2021,3,Summer,30135,"Allentown/Bethlehem/Easton, PA",10135,ABE,Lehigh Valley International Airport,ABE - Lehigh Valley International Airport,Pennsylvania,...,DL,Delta Air Lines,DL - Delta Air Lines,0.4659,219.98,UA,United Airlines,UA - United Airlines,0.1193,154.11
2,2021,3,Summer,30140,"Albuquerque, NM",10140,ABQ,Albuquerque International Sunport,ABQ - Albuquerque International Sunport,New Mexico,...,WN,Southwest Airlines,WN - Southwest Airlines,0.9968,184.44,WN,Southwest Airlines,WN - Southwest Airlines,0.9968,184.44
3,2021,3,Summer,30140,"Albuquerque, NM",10140,ABQ,Albuquerque International Sunport,ABQ - Albuquerque International Sunport,New Mexico,...,AA,American Airlines,AA - American Airlines,0.9774,183.09,AA,American Airlines,AA - American Airlines,0.9774,183.09
4,2021,3,Summer,30140,"Albuquerque, NM",10140,ABQ,Albuquerque International Sunport,ABQ - Albuquerque International Sunport,New Mexico,...,WN,Southwest Airlines,WN - Southwest Airlines,0.6061,184.49,AA,American Airlines,AA - American Airlines,0.3939,165.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245950,2024,1,Winter,35412,"Knoxville, TN",15412,TYS,McGhee Tyson Airport,TYS - McGhee Tyson Airport,Tennessee,...,DL,Delta Air Lines,DL - Delta Air Lines,0.7503,287.44,AA,American Airlines,AA - American Airlines,0.2359,248.46
245951,2024,1,Winter,35412,"Knoxville, TN",15412,TYS,McGhee Tyson Airport,TYS - McGhee Tyson Airport,Tennessee,...,G4,Allegiant Air,G4 - Allegiant Air,0.8255,114.45,G4,Allegiant Air,G4 - Allegiant Air,0.8255,114.45
245952,2024,1,Winter,35412,"Knoxville, TN",15412,TYS,McGhee Tyson Airport,TYS - McGhee Tyson Airport,Tennessee,...,AA,American Airlines,AA - American Airlines,0.8057,321.92,AA,American Airlines,AA - American Airlines,0.8057,321.92
245953,2024,1,Winter,35412,"Knoxville, TN",15412,TYS,McGhee Tyson Airport,TYS - McGhee Tyson Airport,Tennessee,...,G4,Allegiant Air,G4 - Allegiant Air,1.0000,95.65,G4,Allegiant Air,G4 - Allegiant Air,1.0000,95.65


# Data Preproecessing

In [5]:
df.columns

Index(['year', 'quarter', 'season', 'citymarketid_1', 'city1', 'airportid_1',
       'airport_iata_1', 'airport_name_1', 'airport_name_concat_1', 'state_1',
       'latitude_1', 'longitude_1', 'citymarketid_2', 'city2', 'airportid_2',
       'airport_iata_2', 'airport_name_2', 'airport_name_concat_2', 'state_2',
       'latitude_2', 'longitude_2', 'nsmiles', 'passengers', 'fare',
       'carrier_lg', 'carrier_lg_name', 'carrier_lg_name_concat', 'large_ms',
       'fare_lg', 'carrier_low', 'carrier_low_name', 'carrier_low_name_concat',
       'lf_ms', 'fare_low'],
      dtype='object')

In [6]:
# Select column/feature names that we will use to train with
selected_feature_names = ['airport_name_concat_1', 'airport_name_concat_2', 'season', 'year', 'fare']

In [7]:
df = df.loc[:,selected_feature_names]
print(df.shape)


(245955, 5)


In [8]:
df

,airport_name_concat_1,airport_name_concat_2,season,year,fare
0,ABE - Lehigh Valley International Airport,PIE - St. Pete-Clearwater International Airport,Summer,2021,81.43
1,ABE - Lehigh Valley International Airport,TPA - Tampa International Airport,Summer,2021,208.93
2,ABQ - Albuquerque International Sunport,DAL - Dallas Love Field,Summer,2021,184.56
3,ABQ - Albuquerque International Sunport,DFW - Dallas/Fort Worth International Airport,Summer,2021,182.64
4,ABQ - Albuquerque International Sunport,PHX - Phoenix Sky Harbor International Airport,Summer,2021,177.11
...,...,...,...,...,...
245950,TYS - McGhee Tyson Airport,LGA - LaGuardia Airport,Winter,2024,278.70
245951,TYS - McGhee Tyson Airport,FLL - Fort Lauderdale-Hollywood International ...,Winter,2024,148.69
245952,TYS - McGhee Tyson Airport,MIA - Miami International Airport,Winter,2024,330.19
245953,TYS - McGhee Tyson Airport,PIE - St. Pete-Clearwater International Airport,Winter,2024,95.65


In [9]:
# One-hot encode airport_name_concat_1	
unique_values = df['airport_name_concat_1'].unique()
print('Number of unique airport_name_concat_1 values: ', unique_values.shape[0])
df = pd.get_dummies(df, columns=['airport_name_concat_1'], prefix='airport_name_concat_1')
print(df.shape)

Number of unique airport_name_concat_1 values:  177
(245955, 181)


In [10]:
# One-hot encode airport_name_concat_2
unique_values = df['airport_name_concat_2'].unique()
print('Number of unique airport_name_concat_2 values: ', unique_values.shape[0])
df = pd.get_dummies(df, columns=['airport_name_concat_2'], prefix='airport_name_concat_2')
print(df.shape)

Number of unique airport_name_concat_2 values:  166
(245955, 346)


In [11]:
# One-hot encode season
unique_values = df['season'].unique()
print('Number of unique season values: ', unique_values.shape[0])
df = pd.get_dummies(df, columns=['season'], prefix='season')
print(df.shape)

Number of unique season values:  4
(245955, 349)


In [12]:
df

,year,fare,airport_name_concat_1_ABE - Lehigh Valley International Airport,airport_name_concat_1_ABQ - Albuquerque International Sunport,airport_name_concat_1_ACK - Nantucket Memorial Airport,airport_name_concat_1_ACV - Arcata-Eureka Airport,airport_name_concat_1_ACY - Atlantic City International Airport,airport_name_concat_1_AGS - Augusta Regional Airport at Bush Field,airport_name_concat_1_ALB - Albany International Airport,airport_name_concat_1_AMA - Rick Husband Amarillo International Airport,...,airport_name_concat_2_TVC - Cherry Capital Airport,airport_name_concat_2_TYS - McGhee Tyson Airport,airport_name_concat_2_VGT - North Las Vegas Airport,airport_name_concat_2_VPS - Destin-Fort Walton Beach Airport / Eglin Air Force Base,airport_name_concat_2_VRB - Vero Beach Regional Airport,airport_name_concat_2_XNA - Northwest Arkansas Regional Airport,season_Fall,season_Spring,season_Summer,season_Winter
0,2021,81.43,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,2021,208.93,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,2021,184.56,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,2021,182.64,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,2021,177.11,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245950,2024,278.70,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
245951,2024,148.69,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
245952,2024,330.19,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
245953,2024,95.65,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [13]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Data Train Test Split

In [14]:
X = df.drop('fare', axis=1)
y = df['fare']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(196764, 348)
(196764,)
(49191, 348)
(49191,)


In [16]:
X_train

,year,airport_name_concat_1_ABE - Lehigh Valley International Airport,airport_name_concat_1_ABQ - Albuquerque International Sunport,airport_name_concat_1_ACK - Nantucket Memorial Airport,airport_name_concat_1_ACV - Arcata-Eureka Airport,airport_name_concat_1_ACY - Atlantic City International Airport,airport_name_concat_1_AGS - Augusta Regional Airport at Bush Field,airport_name_concat_1_ALB - Albany International Airport,airport_name_concat_1_AMA - Rick Husband Amarillo International Airport,airport_name_concat_1_ASE - Aspen-Pitkin County Airport (Sardy Field),...,airport_name_concat_2_TVC - Cherry Capital Airport,airport_name_concat_2_TYS - McGhee Tyson Airport,airport_name_concat_2_VGT - North Las Vegas Airport,airport_name_concat_2_VPS - Destin-Fort Walton Beach Airport / Eglin Air Force Base,airport_name_concat_2_VRB - Vero Beach Regional Airport,airport_name_concat_2_XNA - Northwest Arkansas Regional Airport,season_Fall,season_Spring,season_Summer,season_Winter
4506,2022,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
189419,2016,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
50467,2000,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
75080,2003,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
58490,2012,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1993,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
103694,2001,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
131932,1994,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
146867,1999,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [17]:
# Save the columns used in the model
model_columns = X_train.columns
joblib.dump(model_columns, 'decision_and_forest_model_columns.pkl')  # Save to file

['decision_and_forest_model_columns.pkl']

# Decision Tree Regressor

### Train & Evaluate Example

In [18]:
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)


DecisionTreeRegressor(random_state=42)

In [19]:
y_pred = regressor.predict(X_test)

# Using Mean Squared Error (MSE) to measure the performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 3586.7236821375977


In [20]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Mean Absolute Percentage Error (MAPE): 13.718526811380732%


### Hyperparameter Tuning Grid Search

In [22]:
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 50, 75, 100, 150],
    'min_samples_leaf': [1, 2, 5, 10]
}
    
regressor = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=5, 
                           scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
best_regressor = grid_search.best_estimator_

Fitting 5 folds for each of 168 candidates, totalling 840 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   4.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   4.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   4.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   4.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   4.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   4.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   4.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   4.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=20; total time=   4

In [25]:
results = pd.DataFrame(grid_search.cv_results_)

columns_of_interest = [
    'param_max_depth', 
    'param_min_samples_split', 
    'param_min_samples_leaf',
    'mean_test_score',  # this is negative MSE, so lower is better
    'rank_test_score'
]
results[columns_of_interest].sort_values(by='mean_test_score', ascending=False)

# Save the selected columns to a CSV file
results[columns_of_interest].to_csv("decision_tree_gridsearch_results.csv", index=False)

In [64]:
# Best parameters: {{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 20}

# Instantiate the regressor with the best parameters
best_regressor = DecisionTreeRegressor(
    max_depth=None,  # or the specific value found in your grid search if it wasn't None
    min_samples_leaf=1,
    min_samples_split=20,
    random_state=42
)

# Train the regressor
best_regressor.fit(X_train, y_train)


DecisionTreeRegressor(min_samples_split=20, random_state=42)

In [65]:
# Train Error
print("Train:")
y_pred = best_regressor.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_train, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")


# Test Error
print("Test:")
y_pred = best_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Train:
Mean Squared Error: 1513.1285780431047
Mean Absolute Percentage Error (MAPE): 9.19897999485746%
Test:
Mean Squared Error: 2516.1740750671825
Mean Absolute Percentage Error (MAPE): 12.883669170472519%


In [66]:
joblib.dump(best_regressor, 'best_decision_tree_regressor.pkl')


['best_decision_tree_regressor.pkl']

# Random Forest Regressor

In [26]:
from sklearn.ensemble import RandomForestRegressor 


### Train & Evaluate Example

In [27]:
# Initialize the RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Mean Squared Error: 2378.7630706332425
Mean Absolute Percentage Error (MAPE): 11.310980096554538%


### Hyperparameter Grid Search

In [28]:
# Define the parameter grid
param_grid = {
    'n_estimators': [30, 50, 100, 200, 300],
    'max_features':['sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10,  50, 75, 100],
    'min_samples_leaf': [1, 2, 5, 10]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", -grid_search.best_score_)

best_regressor = grid_search.best_estimator_

Fitting 3 folds for each of 960 candidates, totalling 2880 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  15.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=  16.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   4.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=  16.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=50, n_estimators=50; total time=   4.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=50, n_estimators=100; total time=   8.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=50, n_estimator

/opt/anaconda3/envs/DataAnalysisProject/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  41.2s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=30; total time=   3.7s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=30; total time=   3.8s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=30; total time=   3.7s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   6.2s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=  11.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=  35.2s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=75, n_estimators=30; total time=   3.4s
[CV] END max_depth=20, max_features=lo

In [29]:
# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", -grid_search.best_score_)

Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Best score: 2570.0796637934636


In [32]:
forest_results = pd.DataFrame(grid_search.cv_results_)

columns_of_interest = [
    'param_max_depth', 
    'param_min_samples_split', 
    'param_min_samples_leaf',
    'param_n_estimators',
    'param_max_features',
    'mean_test_score',  # this is negative MSE, so lower is better
    'rank_test_score'
]
forest_results[columns_of_interest].sort_values(by='mean_test_score', ascending=False)

# Save the selected columns to a CSV file
forest_results[columns_of_interest].to_csv("forest_gridsearch_results.csv", index=False)

In [34]:
# Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}

# overall trend 100-200 is good, 300 pleateaus and is unncessary. 

best_regressor = RandomForestRegressor(
    max_depth=None, 
    max_features = 'sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators = 100,
    random_state=42
)

# Train the regressor
best_regressor.fit(X_train, y_train)

RandomForestRegressor(max_features='sqrt', min_samples_split=5, random_state=42)

In [35]:
# Train Error
print("Train:")
y_pred = best_regressor.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_train, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")


# Test Error
print("Test:")
y_pred = best_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Train:
Mean Squared Error: 1059.4054153373575
Mean Absolute Percentage Error (MAPE): 8.570414855002886%
Test:
Mean Squared Error: 2384.133880946629
Mean Absolute Percentage Error (MAPE): 13.4488770223433%


In [36]:
# Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}

# quite large of a model, so lets do 30 instead

best_regressor = RandomForestRegressor(
    max_depth=None, 
    max_features = 'sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators = 30,
    random_state=42
)

# Train the regressor
best_regressor.fit(X_train, y_train)

RandomForestRegressor(max_features='sqrt', min_samples_split=5, n_estimators=30,
                      random_state=42)

In [37]:
# Train Error
print("Train:")
y_pred = best_regressor.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_train, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")


# Test Error
print("Test:")
y_pred = best_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Train:
Mean Squared Error: 1096.618309810112
Mean Absolute Percentage Error (MAPE): 8.773340458430534%
Test:
Mean Squared Error: 2439.1640076099834
Mean Absolute Percentage Error (MAPE): 13.687759401932173%


In [95]:
joblib.dump(best_regressor, 'best_random_forest_regressor.pkl')


['best_random_forest_regressor.pkl']